In [210]:
import requests as req
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import feedparser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pymongo import MongoClient
import time
import tqdm
import warnings
warnings.filterwarnings('ignore')

PATH = ChromeDriverManager().install()

In [211]:
client = MongoClient('mongodb://localhost:27017')
db = client['hoteles']
collection = db['hoteles-mad']

In [239]:
driver = webdriver.Chrome(PATH)

In [243]:
driver.get('https://www.tripadvisor.es/Hotels-g187514-oa870-Madrid-Hotels.html')

In [241]:
#aceptar cookies
wait = WebDriverWait(driver, 15)
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))).click()

In [242]:
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="component_3"]/div/button/span').click()
driver.execute_script('window.scrollTo(0, 900);')

In [248]:
contador = 0
while 1:
    
    ls = driver.find_elements(By.CLASS_NAME, 'property_title.prominent')
    
    for e in ls:
        
        time.sleep(2)
        
        try:
            try:
                e.click()

            except:
                driver.execute_script('window.scrollBy(0, 250);')
                e.click()
        except:
            driver.execute_script('window.scrollBy(0, 150);')
            e.click()
            
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[1])
        driver.execute_script("return document.readyState == 'complete';")

        try:
            name = driver.find_element(By.CLASS_NAME, 'QdLfr.b.d.Pn').text
        except:
            driver.refresh()
            name = driver.find_element(By.CLASS_NAME, 'QdLfr.b.d.Pn').text

        try:
            ubicacion = driver.find_element(By.CLASS_NAME, 'fHvkI.PTrfg').text
        except:
            ubicacion = driver.find_element(By.XPATH, '//*[@id="component_3"]/div/div/div[2]/div/div[2]/div/div/div/span[2]/span').text
        
        try:
            link = driver.find_element(By.CLASS_NAME, 'YnKZo.Ci.Wc._S.C.pInXB._S.ITocq.jNmfd').get_attribute('href')
        except:
            link = 'No hay link'
            
        try:
            try:
                try:
                    precio = driver.find_element(By.CLASS_NAME, 'JPNOn.b.Wi').text
                except:
                    precio = driver.find_element(By.CLASS_NAME, 'WXMFC.b').text

            except:
                precio = driver.find_element(By.CLASS_NAME, 'WXMFC.b.autoResize').text
        except:
            precio = 'No disponible'
        
        
        driver.execute_script('window.scrollTo(0, 700);')
        try:
            puntuacion = driver.find_element(By.CLASS_NAME, 'uwJeR.P').text 
        except:
            puntuacion = 'Este alojamiento todavía no tiene valoraciones, ¡se el primero!'
        
        driver.execute_script('window.scrollTo(0, 1000);')
        time.sleep(0.5)
        try:
            descripcion = driver.find_element(By.CLASS_NAME, 'fIrGe._T').text
        except:
            descripcion = 'No disponible'
        
        time.sleep(1)
        
        serv_hab = []
        tipo_hab = []
        dos = driver.find_elements(By.CLASS_NAME, 'yplav.f.ME.H3._c')
        
        for e in dos:
            if 'Suites' in e.text or 'Habitaciones' in e.text or 'Vistas' in e.text:
                tipo_hab.append(e.text)
            elif e.text == '':
                pass
            else:
                serv_hab.append(e.text)
        
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

        hoteles = {'_id':name, 'nombre': name, 'ubicacion':ubicacion, 'link':link, 'precio':precio,
                   'puntuacion':puntuacion,'descripcion':descripcion, 'serv_hab': serv_hab, 'tipo_hab': tipo_hab}
        
        print(name, ubicacion)
        
        try:
            collection.insert_one(hoteles)
            serv_hab.clear()
            tipo_hab.clear()
            contador += 1
            print(contador)

        except:
            continue
    
    try:
        driver.find_element(By.CLASS_NAME, 'nav.next.ui_button.primary').click()
        time.sleep(7)
    except:
        break

Canopy by Hilton Madrid Castellana Plaza Carlos Trias Bertran 4, 28020 Madrid España
Hostal Las Fuentes Calle Fuentes 10 1 D Plaza de la Opera, 28013 Madrid España
1
L&H Gran Vía Selection San Onofre 5, 28004 Madrid España
2
Hotel Gran Legazpi Paseo de la Chopera, 71 BAJO;HOTEL GRAN LEGAZPI, 28045 Madrid España
3
Hostal Benamar Calle San Mateo 20 nº 20 2º y 3º Dcha., 28004 Madrid España
4
Air Rooms Madrid Airport by Premium Traveller Aeropuerto de Madrid, Barajas, Terminal T4, Planta -1 (Zona tierra), 28042 Madrid España
5
Pension Jaen Calle Cervantes 5, 28014 Madrid España
6
Senator Gran Vía 70 Spa Hotel Calle Gran Via 70, 28015 Madrid España
7
Occidental Madrid Este Calle Julian Camarillo 19-21, 28037 Madrid España
8
Hotel Arosa Calle Salud, 21, 28013 Madrid España
9
TRYP Madrid Chamberí Hotel Calle Jose Abascal 8, 28003 Madrid España
10
Hotel Trafalgar Trafalgar 35, 28010 Madrid España
11
Hotel Miguel Angel Calle Miguel Angel 29-31, 28010 Madrid España
12
RC Miguel Angel Plaza de Ce

KeyboardInterrupt: 

### A PARTIR DE AQUÍ NO UTILIZAR

In [246]:
driver.close()
driver.switch_to.window(driver.window_handles[0])